In [1]:
!pip3 install catboost
!pip3 install hyperopt==0.2.5

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool, metrics, cv
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import hyperopt
from numpy.random import RandomState
from catboost.utils import select_threshold

In [43]:
csv_path = "drive/MyDrive/trades-df-2021_11_1_0_0_0-2021_12_21_0_0_0.csv"
df = pd.read_csv(csv_path)
x_columns = list(filter(lambda column: not 'punch' in column, df.columns))
y_columns = list(filter(lambda column: 'punch' in column, df.columns))
X = df[x_columns]
secs = list(sorted(list(set(map(lambda x : int(''.join(x)), [[s for s in column if s.isdigit()] for column in x_columns])))))
delta_sec = secs[1] - secs[0]
max_sec = secs[-1]
min_sec = secs[0]

for column in x_columns:
    name, side, sec, _ = column.split('-')
    if not 'price' in name and int(sec) != min_sec:
        X[f'{name}-{side}-{sec}-sec'] -= X[f'{name}-{side}-{str(int(sec) - delta_sec)}-sec']
    elif 'price' in name:
        if int(sec) != max_sec:
            X[f'{name}-{side}-{sec}-sec'] /= X[f'{name}-{side}-{str(int(sec) + delta_sec)}-sec']
        else:
            X = X.drop(f'{name}-{side}-{sec}-sec', axis=1)

commision = 0.0005
punch = commision * 4

y = pd.DataFrame(
    df[y_columns].apply(
        lambda row : 1 if max(list(row.to_numpy()), key=abs) > punch else -1 if max(list(row.to_numpy()), key=abs) < -punch else 0,
        axis = 1
    )
)
y = y.rename({0:'target'}, axis='columns')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
print(y.value_counts())
y

target
 0        36607
-1         1155
 1          985
dtype: int64


,target
0,0
1,0
2,1
3,0
4,0
...,...
38742,0
38743,0
38744,0
38745,0


In [45]:
X

,trade_volume-BUY-6-sec,moving_average_price-BUY-6-sec,weighted_moving_average_price-BUY-6-sec,exp_moving_average_price-BUY-6-sec,stochastic_oscillator-BUY-6-sec,trade_volume-BUY-12-sec,moving_average_price-BUY-12-sec,weighted_moving_average_price-BUY-12-sec,exp_moving_average_price-BUY-12-sec,stochastic_oscillator-BUY-12-sec,trade_volume-BUY-18-sec,moving_average_price-BUY-18-sec,weighted_moving_average_price-BUY-18-sec,exp_moving_average_price-BUY-18-sec,stochastic_oscillator-BUY-18-sec,trade_volume-BUY-24-sec,moving_average_price-BUY-24-sec,weighted_moving_average_price-BUY-24-sec,exp_moving_average_price-BUY-24-sec,stochastic_oscillator-BUY-24-sec,trade_volume-BUY-30-sec,moving_average_price-BUY-30-sec,weighted_moving_average_price-BUY-30-sec,exp_moving_average_price-BUY-30-sec,stochastic_oscillator-BUY-30-sec,trade_volume-BUY-36-sec,moving_average_price-BUY-36-sec,weighted_moving_average_price-BUY-36-sec,exp_moving_average_price-BUY-36-sec,stochastic_oscillator-BUY-36-sec,trade_volume-BUY-42-sec,moving_average_price-BUY-42-sec,weighted_moving_average_price-BUY-42-sec,exp_moving_average_price-BUY-42-sec,stochastic_oscillator-BUY-42-sec,trade_volume-BUY-48-sec,moving_average_price-BUY-48-sec,weighted_moving_average_price-BUY-48-sec,exp_moving_average_price-BUY-48-sec,stochastic_oscillator-BUY-48-sec,...,weighted_moving_average_price-SELL-12-sec,exp_moving_average_price-SELL-12-sec,stochastic_oscillator-SELL-12-sec,trade_volume-SELL-18-sec,moving_average_price-SELL-18-sec,weighted_moving_average_price-SELL-18-sec,exp_moving_average_price-SELL-18-sec,stochastic_oscillator-SELL-18-sec,trade_volume-SELL-24-sec,moving_average_price-SELL-24-sec,weighted_moving_average_price-SELL-24-sec,exp_moving_average_price-SELL-24-sec,stochastic_oscillator-SELL-24-sec,trade_volume-SELL-30-sec,moving_average_price-SELL-30-sec,weighted_moving_average_price-SELL-30-sec,exp_moving_average_price-SELL-30-sec,stochastic_oscillator-SELL-30-sec,trade_volume-SELL-36-sec,moving_average_price-SELL-36-sec,weighted_moving_average_price-SELL-36-sec,exp_moving_average_price-SELL-36-sec,stochastic_oscillator-SELL-36-sec,trade_volume-SELL-42-sec,moving_average_price-SELL-42-sec,weighted_moving_average_price-SELL-42-sec,exp_moving_average_price-SELL-42-sec,stochastic_oscillator-SELL-42-sec,trade_volume-SELL-48-sec,moving_average_price-SELL-48-sec,weighted_moving_average_price-SELL-48-sec,exp_moving_average_price-SELL-48-sec,stochastic_oscillator-SELL-48-sec,trade_volume-SELL-54-sec,moving_average_price-SELL-54-sec,weighted_moving_average_price-SELL-54-sec,exp_moving_average_price-SELL-54-sec,stochastic_oscillator-SELL-54-sec,trade_volume-SELL-60-sec,stochastic_oscillator-SELL-60-sec
0,0.100,1.000000,1.000000,1.000000,0.0,0.400,1.000078,1.000113,1.000005,0.000000,10.355,1.000006,1.000000,1.000000,1.000000,0.500,1.000013,1.000001,1.000000,0.000000,10.655,1.000020,1.000002,1.0,1.000000,0.716,1.000023,1.000004,1.000000,0.000000,10.855,1.000019,1.000004,1.000000,1.000000,0.916,1.000000,1.000000,1.0,0.000000,...,1.000067,1.000007,0.000000,0.400,1.000035,1.000038,1.000000,1.000000,0.800,1.000006,1.000005,1.000000,0.000000,0.600,1.000035,1.000140,1.0,1.000000,3.500,1.000020,1.000004,1.000000,0.000000,0.800,1.000023,1.000024,1.0,1.000000,5.566,1.000012,1.000014,1.0,0.000000,3.400,1.000028,1.000018,1.0,1.000000,15.547,0.000000
1,0.200,1.000000,1.000000,1.000000,0.0,0.300,1.000000,1.000000,1.000000,0.000000,0.400,1.000000,1.000000,1.000000,0.000000,0.600,1.000000,1.000000,1.000000,0.000000,0.650,1.000000,1.000000,1.0,0.000000,0.800,1.000037,1.000048,1.000000,0.000000,1.399,1.000005,1.000004,1.000000,0.600000,0.900,0.999985,0.999986,1.0,0.000000,...,1.000000,1.000000,0.000000,0.766,1.000000,1.000000,1.000000,0.000000,4.084,1.000000,1.000000,1.000000,0.000000,0.766,1.000008,1.000003,1.0,0.000000,4.284,1.000025,1.000010,1.000000,0.600000,1.166,1.000000,1.000000,1.0,0.000000,4.284,0.999875,0.999760,1.0,0.600000,34.697,0.999995,1.000000,1.0,-0.339130,4.384,0.600000
2,0.200,1.0

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, shuffle = True, random_state=12)
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, train_size=0.75, shuffle = True, random_state=6)
params = {
    'iterations': 500,
    'l2_leaf_reg': 2,
    'learning_rate': 0.1,
    'custom_loss': [metrics.Accuracy()],
    'eval_metric': metrics.Accuracy(),
    'random_seed': 42,
    'depth': 5,
    'logging_level': 'Silent',
    'loss_function': 'MultiClass',
}
train_pool = Pool(X_train, y_train)
validate_pool = Pool(X_validation, y_validation)

model = CatBoostClassifier(**params)
model.fit(
    train_pool,
    eval_set=validate_pool,
    plot=True,
    verbose=False,
)

print('Model validation accuracy: {:.4}'.format(
    accuracy_score(y_test, model.predict(X_test))
))
predictions = model.predict(X_test)
predictions = predictions.reshape(predictions.shape[0], 1)
predictions_probs = model.predict_proba(X_test)
print(predictions[90:100])
print(predictions_probs[90:100])
print(y_test[90:100])
feature_importances = model.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Model validation accuracy: 0.9484
[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]]
[[0.24832678 0.50375411 0.24791912]
 [0.05329057 0.90007009 0.04663934]
 [0.03027111 0.94011954 0.02960935]
 [0.01977515 0.96185938 0.01836547]
 [0.03616962 0.93580581 0.02802457]
 [0.02726176 0.94901081 0.02372742]
 [0.08373739 0.83054342 0.0857192 ]
 [0.02565453 0.95194086 0.02240461]
 [0.02029947 0.96105465 0.01864588]
 [0.04872791 0.90892067 0.04235141]]
       target
20233      -1
18782       0
26657       0
27456       0
36872       0
25891       0
33455      -1
23349       0
4176        0
37949       0
moving_average_price-SELL-6-sec: 5.61635480466443
trade_volume-SELL-54-sec: 5.514494280220464
moving_average_price-BUY-12-sec: 4.895678495947581
trade_volume-BUY-48-sec: 4.257752784909689
trade_volume-SELL-36-sec: 3.9714158974601013
trade_volume-SELL-24-sec: 3.86443870826791
moving_average_price-BUY-48-sec: 3.2892424784294616
trade_volume-BUY-54-sec: 3.2567574371768107
weighted_moving_average_pric

In [47]:
def calculate_profit(prob_threshold, diff_threshold, null_threshold):
    p = f(prob_threshold, diff_threshold, null_threshold)
    result = {
        'correct_predictions': 0,
        'false_positive': 0,
        'false_negative': 0,
        'wrong_side': 0,
    }
    for i in range(len(p)):
        t = int(y_test['target'][i:i+1])
        if p[i] == t != 0:
            result['correct_predictions'] += 1
        elif p[i] != t and t != 0 and p[i] != 0:
            result['wrong_side'] += 1
        elif p[i] != 0 and t == 0:
            result['false_positive'] += 1
        elif p[i] == 0 and t != 0:
            result['false_negative'] += 1

    result['correct_predictions_pc'] = result['correct_predictions'] / len(y_test)
    result['false_positive_pc'] = result['false_positive'] / len(y_test)
    result['false_negative_pc'] = result['false_negative'] / len(y_test)
    result['wrong_side_pc'] = result['wrong_side'] / len(y_test)
    
    return (
          len(y) * result['correct_predictions_pc'] * (punch - 2 * commision)
          -
          len(y) * result['false_positive_pc'] * 2 * commision
          -
          len(y) * result['wrong_side_pc'] * (punch + 2 * commision)
    )

results = []
def f(prob_threshold, diff_threshold, null_threshold):
    p = np.array((list(map(lambda row:
                          1
                          if row[2] > prob_threshold
                              # and row[2] == max(row)
                              and row[2] > row[0]
                              and row[1] < null_threshold
                              and abs(row[2] - row[0]) > diff_threshold
                              and abs(row[2] - row[1]) > diff_threshold
                          else
                          -1
                          if row[0] > prob_threshold
                              # and row[0] == max(row)
                              and row[0] > row[2]
                              and row[1] < null_threshold
                              and abs(row[0] - row[2]) > diff_threshold
                              and abs(row[0] - row[1]) > diff_threshold
                          else 0,
                          predictions_probs))))
    result = {
        'correct_predictions': 0,
        'false_positive': 0,
        'false_negative': 0,
        'wrong_side': 0,
    }
    for i in range(len(p)):
        t = int(y_test['target'][i:i+1])
        if p[i] == t != 0:
            result['correct_predictions'] += 1
        elif p[i] != t and t != 0 and p[i] != 0:
            result['wrong_side'] += 1
        elif p[i] != 0 and t == 0:
            result['false_positive'] += 1
        elif p[i] == 0 and t != 0:
            result['false_negative'] += 1

    result['correct_predictions'] /= len(y_test)
    result['false_positive'] /= len(y_test)
    result['false_negative'] /= len(y_test)
    result['wrong_side'] /= len(y_test)
    result['prob_threshold'] = prob_threshold
    result['diff_threshold'] = diff_threshold
    result['null_threshold'] = null_threshold
    results.append(result)
    return p

In [48]:
for null_threshold in np.arange(0.5, 0.8, 0.1):
    for prob_threshold in np.arange(0.3, 0.7, 0.1):
        for diff_threshold in np.arange(0.00, 0.20, 0.05):
            f(prob_threshold, diff_threshold, null_threshold)

In [49]:
results = list(sorted(results, key=lambda result: -calculate_profit(result['prob_threshold'], result['diff_threshold'], result['null_threshold'])))
print(results[0])

{'correct_predictions': 0.0007741935483870968, 'false_positive': 0.0, 'false_negative': 0.05174193548387097, 'wrong_side': 0.0, 'prob_threshold': 0.4, 'diff_threshold': 0.1, 'null_threshold': 0.5}


In [50]:
prob_threshold = results[0]['prob_threshold']
diff_threshold = results[0]['diff_threshold']
null_threshold = results[0]['null_threshold']

print(f"prob_threshold {prob_threshold}, diff_threshold {diff_threshold}, null_threshold {null_threshold}")
p = f(prob_threshold, diff_threshold, null_threshold)
print('accuracy', accuracy_score(p, y_test))
result = {
    'correct_predictions': 0,
    'false_positive': 0,
    'false_negative': 0,
    'wrong_side': 0,
}
for i in range(len(p)):
    t = int(y_test['target'][i:i+1])
    if p[i] == t != 0:
        result['correct_predictions'] += 1
    elif p[i] != t and t != 0 and p[i] != 0:
        result['wrong_side'] += 1
    elif p[i] != 0 and t == 0:
        result['false_positive'] += 1
    elif p[i] == 0 and t != 0:
        result['false_negative'] += 1

result['correct_predictions_pc'] = result['correct_predictions'] / len(y_test)
result['false_positive_pc'] = result['false_positive'] / len(y_test)
result['false_negative_pc'] = result['false_negative'] / len(y_test)
result['wrong_side_pc'] = result['wrong_side'] / len(y_test)
print(result)

prob_threshold 0.4, diff_threshold 0.1, null_threshold 0.5
accuracy 0.948258064516129
{'correct_predictions': 6, 'false_positive': 0, 'false_negative': 401, 'wrong_side': 0, 'correct_predictions_pc': 0.0007741935483870968, 'false_positive_pc': 0.0, 'false_negative_pc': 0.05174193548387097, 'wrong_side_pc': 0.0}


In [51]:
profit = calculate_profit(prob_threshold, diff_threshold, null_threshold)
print(
    'profit from 3000$ fot 2 months',
    profit * 3000,
    '$',
    profit * 100,
    '%'
)

profit from 3000$ fot 2 months 89.9930322580645 $ 2.9997677419354836 %


In [52]:
correct_accuracy = result['correct_predictions'] / len(y_test[y_test['target'] != 0])
print(
f"Correct accuracy = correct prediction / total positive num = {correct_accuracy}"
)
FPR = result['false_positive'] / len(y_test[y_test['target'] == 0])
print(
f"FPR = false positive / total negative num = {FPR}"
)
FNR = result['false_negative'] / len(y_test[y_test['target'] != 0])
print(
f"FNR = false negative / total positive num = {FNR}"
)
cases_pc = (result['correct_predictions'] + result['false_positive'] + result['wrong_side']) / len(y_test[y_test['target'] != 0]) * 100
probability_pc = result['correct_predictions'] / (result['correct_predictions'] + result['false_positive'] + result['wrong_side']) * 100
print(
f"Model in {cases_pc}% of cases with probability {probability_pc}% predict a correct market jump"
)

Correct accuracy = correct prediction / total positive num = 0.014742014742014743
FPR = false positive / total negative num = 0.0
FNR = false negative / total positive num = 0.9852579852579852
Model in 1.4742014742014742% of cases with probability 100.0% predict a correct market jump
